# Matrix Factorization: A Simple Tutorial and Implementation in Python

http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/

## Basic Ideas
From an application point of view, matrix factorization can be used to discover latent features underlying the interactions between two different kinds of entities.

In a recommendation system such as Netflix or MovieLens, there is a group of `users` and a set of `items` (movies for the above two systems). Given that each users have rated some items in the system, we would like to predict how the users would rate the items that they have not yet rated, such that we can make recommendations to the users.

Assume now we have 5 users and 4 items, and ratings are integers ranging from 1 to 5:

| item/user | D1 | D2 | D3 | D4 |
| ----- | ----- | ----- | ----- | ----- |
| U1 | 5 | 3 | - | 1 |
| U2 | 4 | - | - | 1 |
| U3 | 1 | 1 | - | 5 |
| U4 | 1 | - | - | 4 |
| U5 | - | 1 | 5 | 4 |

Hence, the task of predicting the missing ratings can be considered as filling in the blanks.

## The mathematics of matrix factorization

- Firstly, we have a set U of users, and a set D of items. Let R of size |U|x|D| be the matrix that contains all the ratings that the users have assigned to the items.
- Also, we assume that we would like to discover K latent features. Our task, then, is to find two matrics matrices P (a |U|xK matrix) and Q(a |D|xK matrix) such that their product approximates R: 

$R\approx P \times Q^T = \hat{R}$

- Each row of P would represent the strength of the associations between a user and the features.
- Each row of Q would represent the strength of the associations between an item and the features.
- To get the prediction of a rating of an item $d_j$ by $u_i$, we can calculate the dot product of the two vectors corresponding to $u_i$ and $d_j$:

$\hat{r_{ij}} = p_i^Tq_j = \sum_{k=1}^k p_{ik}q_{kj}$

Error estimate:

$e_{ij}^2 = (r_{ij} - \hat{r_{ij}})^2 = (r_{ij} - \sum_{k=1}^K p_{ik}q_{kj})^2$

To minimize the error, we have to know in which direction we have to modify the values of $p_{ik}$ and $q_{kj}$:

$\frac{\partial}{\partial{p_{ik}}}e_{ij}^2 = -2(r_{ij}-\hat{r_{ij}})(q_{kj}) = -2e_{ij}q_{kj}$

$\frac{\partial}{\partial{q_{ik}}}e_{ij}^2 = -2(r_{ij}-\hat{r_{ij}})(p_{ik}) = -2e_{ij}p_{ik}$

Having obtained the gradient, we can now formulate the update rules for both $p_{ik}$ and $q_{kj}$:

$p_{ik}' = p_{ik} + \alpha \frac{\partial}{\partial{p_{ik}}}e_{ij}^2 = p_{ik} + 2\alpha e_{ij}q_{kj}$

$q_{kj}' = q_{kj} + \alpha \frac{\partial}{\partial{p_{kj}}}e_{ij}^2 = q_{kj} + 2\alpha e_{ij}q_{ik}$

$\alpha$:learning rate

We can then iteratively perform the operation until the error converges to its minimum. We can check the overall error:

$E = \sum_{(u_i,d_j,r_{ij}) \in T}e_{ij} = \sum_{(u_i,d_j,r_{ij}) \in T}(r_{ij} - \sum_{k=1}^K p_{ik}q_{kj})^2$

## Regularization
A common extension to this basic algorithm is to introduce regularization to avoid overfitting:

$e_{ij}^2 = (r_{ij} - \hat{r_{ij}})^2 = (r_{ij} - \sum_{k=1}^K p_{ik}q_{kj})^2 + \frac{\beta}{2}\sum_{k=1}^K(\|P\|^2 + \|Q\|^2)$

$\beta$ is used to control the magnitudes of the user-feature and item-feature vectors such that P and Q would give a good approximation of R without having to contain large numbers.

The new update rules are as follows:

$p_{ik}' = p_{ik} + \alpha \frac{\partial}{\partial{p_{ik}}}e_{ij}^2 = p_{ik} + \alpha(2e_{ij}q_{kj} - \beta p_{ik})$

$q_{kj}' = q_{kj} + \alpha \frac{\partial}{\partial{p_{kj}}}e_{ij}^2 = q_{kj} + \alpha(2e_{ij}q_{ik} - \beta q_{kj})$

## Implementation in Python

In [1]:
#!/usr/bin/python
import numpy as np

###############################################################################

"""
@INPUT:
    R     : a matrix to be factorized, dimension N x M
    P     : an initial matrix of dimension N x K
    Q     : an initial matrix of dimension M x K
    K     : the number of latent features
    steps : the maximum number of steps to perform the optimisation
    alpha : the learning rate
    beta  : the regularization parameter
@OUTPUT:
    the final matrices P and Q
"""
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e += pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in xrange(K):
                        e += (beta/2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break
    return P, Q.T

In [2]:
R = [
     [5,3,0,1],
     [4,0,0,1],
     [1,1,0,5],
     [1,0,0,4],
     [0,1,5,4],
    ]

R = np.array(R)

N = len(R)
M = len(R[0])
K = 2

P = np.random.rand(N, K)  # shape (5, 2)
Q = np.random.rand(M, K)  # shape (4, 2)

nP, nQ = matrix_factorization(R, P, Q, K)
print np.dot(nP, nQ.T)

[[ 5.06764261  2.72043466  5.83556815  0.99861472]
 [ 3.91441045  2.10130191  4.60677696  1.00374907]
 [ 1.13440717  0.60796713  3.33152155  4.96801169]
 [ 0.93310611  0.50010531  2.69581378  3.98212046]
 [ 2.91771876  1.56557017  4.82846645  4.01547552]]
